In [1]:
!pip install gdown
!pip install pandarallel

In [2]:
import argparse
import glob
import os
import json
import time
import logging
import random
import re
from itertools import chain
from string import punctuation

import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset, Dataset
from tqdm import tqdm
import datasets

from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True,nb_workers=8)
tqdm.pandas()

from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [3]:
from transformers import (
    T5ForConditionalGeneration, T5Tokenizer, 
    TrainingArguments, Trainer, DataCollatorForSeq2Seq,AutoTokenizer,FlaxT5ForConditionalGeneration,AutoModelForSeq2SeqLM
  )


In [4]:
url='https://drive.google.com/drive/folders/1QvhirnOFoLqPZYeBz8M9oMZh-hxnOCnt?usp=sharing'

In [5]:
import gdown
gdown.download_folder(url, quiet=True)

['/kaggle/working/data_wav2vec2/benchmark.parquet',
 '/kaggle/working/data_wav2vec2/custom_data.parquet',
 '/kaggle/working/data_wav2vec2/data_sharded/test/cache-43f0f09683d0ab82.arrow',
 '/kaggle/working/data_wav2vec2/data_sharded/test/cache-a99e762e72e1079e.arrow',
 '/kaggle/working/data_wav2vec2/data_sharded/test/data-00000-of-00001.arrow',
 '/kaggle/working/data_wav2vec2/data_sharded/test/dataset_info.json',
 '/kaggle/working/data_wav2vec2/data_sharded/test/state.json',
 '/kaggle/working/data_wav2vec2/data_sharded/train/cache-bf7ee38c5d6c7cce.arrow',
 '/kaggle/working/data_wav2vec2/data_sharded/train/cache-c5e24c7182b4fbb4.arrow',
 '/kaggle/working/data_wav2vec2/data_sharded/train/data-00000-of-00005.arrow',
 '/kaggle/working/data_wav2vec2/data_sharded/train/data-00001-of-00005.arrow',
 '/kaggle/working/data_wav2vec2/data_sharded/train/data-00002-of-00005.arrow',
 '/kaggle/working/data_wav2vec2/data_sharded/train/data-00003-of-00005.arrow',
 '/kaggle/working/data_wav2vec2/data_shar

In [6]:
df_custom = pd.read_parquet('/kaggle/working/data_wav2vec2/custom_data.parquet')
df_wav2vec2_all = pd.read_parquet('/kaggle/working/data_wav2vec2/wav2vec2_all.parquet')
df_wav2vec2_masked = pd.read_parquet('/kaggle/working/data_wav2vec2/wav2vec2_masked.parquet')

In [7]:
df_custom = df_custom[['masked_sen','correct_word']]
df_custom

,masked_sen,correct_word
0,আর জামিনের [MASK] করলেন না অর্পিতা,আবেদন
1,অর্পিতা মুখার্জি [MASK],কারাগারেই
2,তাঁর [MASK] সংশয়ের আশঙ্কা রয়েছে,জীবন
3,আদালত শুনানি শেষে পার্থের জামিন আবেদন [MASK] ক...,খারিজ
4,অর্পিতাকেও দিনের জেল [MASK] রাখার আদেশ দেন,হেফাজতে
...,...,...
2992534,রায়ের পর প্রসিকিউটর রেজিয়া সুলতানা প্রথম [MASK...,আলোকে
2992535,খলিলুরের বিরুদ্ধে মুক্তিযুদ্ধকালে [MASK],আটক
2992536,হত্যা ও গণহত্যাসহ মানবতাবিরোধী অপরাধের পাঁচটি ...,আনা
2992537,[MASK] অভিযোগই ট্রাইব্যুনালে প্রমাণিত হয়েছে,পাঁচটি


In [8]:
df_wav2vec2_masked = df_wav2vec2_masked[['masked_wav2vec2','correct_word']]
df_wav2vec2_masked

,masked_wav2vec2,correct_word
0,হাসি [MASK] হয়েই আছে,বিষয়
1,শাক দেশ [MASK],
2,[MASK] জাদে,সহজেই
3,শহজি [MASK],যাওয়া
4,অন্তর্ভুক্ত করেছে [MASK],রাশিয়া
...,...,...
248769,রেডিও বিকিরণের একটি অংশ আপেক্ষিক জেট থেকে আসেয...,পশ্রন
248770,আব্দুল মালেক [MASK] জেলার মুক্তিযুদ্ধের সংগঠক ...,ময়মনসিংহ
248771,তার জয়ের মাধ্যমে প্রথমবারের মতো কোন তৃনমূল কংগ...,পথ্রী
248772,এখানে [MASK] ব্যাং্ক পরিষেবা বর্তমান,ল্বাড


In [9]:
df_custom.rename(columns = {'masked_sen':'input','correct_word':'output'},inplace=True)
df_custom

,input,output
0,আর জামিনের [MASK] করলেন না অর্পিতা,আবেদন
1,অর্পিতা মুখার্জি [MASK],কারাগারেই
2,তাঁর [MASK] সংশয়ের আশঙ্কা রয়েছে,জীবন
3,আদালত শুনানি শেষে পার্থের জামিন আবেদন [MASK] ক...,খারিজ
4,অর্পিতাকেও দিনের জেল [MASK] রাখার আদেশ দেন,হেফাজতে
...,...,...
2992534,রায়ের পর প্রসিকিউটর রেজিয়া সুলতানা প্রথম [MASK...,আলোকে
2992535,খলিলুরের বিরুদ্ধে মুক্তিযুদ্ধকালে [MASK],আটক
2992536,হত্যা ও গণহত্যাসহ মানবতাবিরোধী অপরাধের পাঁচটি ...,আনা
2992537,[MASK] অভিযোগই ট্রাইব্যুনালে প্রমাণিত হয়েছে,পাঁচটি


In [10]:
df_wav2vec2_masked.rename(columns={'masked_wav2vec2':'input','correct_word':'output'},inplace=True)
df_wav2vec2_masked

,input,output
0,হাসি [MASK] হয়েই আছে,বিষয়
1,শাক দেশ [MASK],
2,[MASK] জাদে,সহজেই
3,শহজি [MASK],যাওয়া
4,অন্তর্ভুক্ত করেছে [MASK],রাশিয়া
...,...,...
248769,রেডিও বিকিরণের একটি অংশ আপেক্ষিক জেট থেকে আসেয...,পশ্রন
248770,আব্দুল মালেক [MASK] জেলার মুক্তিযুদ্ধের সংগঠক ...,ময়মনসিংহ
248771,তার জয়ের মাধ্যমে প্রথমবারের মতো কোন তৃনমূল কংগ...,পথ্রী
248772,এখানে [MASK] ব্যাং্ক পরিষেবা বর্তমান,ল্বাড


In [11]:
df = df_custom.append(df_wav2vec2_masked)
df = df.dropna(how='any',axis=0) 
df

,input,output
0,আর জামিনের [MASK] করলেন না অর্পিতা,আবেদন
1,অর্পিতা মুখার্জি [MASK],কারাগারেই
2,তাঁর [MASK] সংশয়ের আশঙ্কা রয়েছে,জীবন
3,আদালত শুনানি শেষে পার্থের জামিন আবেদন [MASK] ক...,খারিজ
4,অর্পিতাকেও দিনের জেল [MASK] রাখার আদেশ দেন,হেফাজতে
...,...,...
248769,রেডিও বিকিরণের একটি অংশ আপেক্ষিক জেট থেকে আসেয...,পশ্রন
248770,আব্দুল মালেক [MASK] জেলার মুক্তিযুদ্ধের সংগঠক ...,ময়মনসিংহ
248771,তার জয়ের মাধ্যমে প্রথমবারের মতো কোন তৃনমূল কংগ...,পথ্রী
248772,এখানে [MASK] ব্যাং্ক পরিষেবা বর্তমান,ল্বাড


In [12]:
df = df.sample(2041313)
df = df.reset_index(drop=True)
df

,input,output
0,বার্তা সংস্থা এএফপি [MASK],বলেছে
1,পুলিশ [MASK] কম্পিউটার জব্দ করে ব্ল্যাকমেলের স...,তাঁর
2,সাক্ষাৎকার নিয়েছেন ইফতেখার [MASK],মাহমুদ
3,বিশ্ববিদ্যালয়ের মাধ্যমে আমার কন্যা [MASK] নিমন...,পূর্বা
4,দেশে প্রতিবছর মে মাস থেকে জুলাই [MASK] সময়কে আ...,পর্যন্ত
...,...,...
2041308,শীতলপাটির মধ্যে সবচেয়ে [MASK] বিক্রি হয় ফুট বা...,বেশি
2041309,[MASK] এজাহারে তিনি উল্লেখ করেন,মামলার
2041310,এ সময় করোনা [MASK] হয়েছে জনের,শনাক্ত
2041311,বৃষ্টি [MASK] না হলে আমেরও কোনো ক্ষতি হবে না,দীর্ঘস্থায়ী


In [13]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.10, shuffle=True)
train_df.shape, test_df.shape

((1837181, 2), (204132, 2))

In [14]:
train_data = Dataset.from_pandas(train_df.sample(1837181))
test_data = Dataset.from_pandas(test_df)

In [15]:
train_data.save_to_disk('train')
test_data.save_to_disk('test')

In [16]:
max_len = 128

In [17]:
model_name = 'flax-community/bengali-t5-base'
tokenizer = AutoTokenizer.from_pretrained(model_name)

Downloading:   0%|          | 0.00/1.44k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.05M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [18]:
print(tokenizer.special_tokens_map.keys())

dict_keys(['eos_token', 'unk_token', 'pad_token', 'additional_special_tokens'])


In [19]:
tokenizer.decode(tokenizer(train_data[0]['input'],padding="max_length", max_length=64,return_attention_mask = True).input_ids)

'যেন তিনি কেবল ঋণের বোঝা বহনকারী [mask] না হন</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>'

In [20]:
def prepare_data(example):
    example["input_ids"] = tokenizer(example['input'], padding="max_length", max_length=64,return_attention_mask = True).input_ids
    example["attention_mask"] = tokenizer(example['input'], padding="max_length", max_length=64,return_attention_mask = True).attention_mask
    example["labels"] = tokenizer(example["output"], padding="max_length", max_length=64).input_ids
    return example

In [21]:
train_data = train_data.map(prepare_data,remove_columns=train_data.column_names)

  0%|          | 0/1837181 [00:00<?, ?ex/s]

In [22]:
test_data = test_data.map(prepare_data)

  0%|          | 0/204132 [00:00<?, ?ex/s]

In [23]:
train_data.cleanup_cache_files()
test_data.cleanup_cache_files()

0

In [24]:
train_data.save_to_disk('train')
test_data.save_to_disk('test')

In [25]:
!rm -rf /kaggle/working/data_wav2vec2

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
